In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

from tqdm import tqdm

# BERT模型分类

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained(
    '../../H/models/huggingface/bert-base-chinese')

## 创建数据集

In [3]:
train_path = "../../H/datasets/THUCNews/train.txt"
dev_path = "../../H/datasets/THUCNews/dev.txt"
test_path = "../../H/datasets/THUCNews/test.txt"

In [28]:
# 加载语料
def load_corpus(path):
    sentences = []
    labels = []
    with open(path, 'r', encoding='UTF-8') as f:
        for line in tqdm(f):
            line = line.strip()
            if not line:
                continue
            sent, label = line.split('\t')
            sentences.append(sent)
            labels.append(int(label))
    return sentences, labels

train_data, train_labels = load_corpus(train_path)

180000it [00:00, 1428599.28it/s]


In [29]:
len(train_data),len(train_labels)

(180000, 180000)

In [30]:
len(set(train_labels))

10

In [6]:
train_data[900], train_labels[900]

('斯里兰卡急派外交部长前往巴基斯坦', 6)

In [7]:
# 向量化
def vectorize(sentences, tokenizer):
    input_ids = []

    for sent in sentences:
        encoded_sent = tokenizer.encode(
            sent,
            add_special_tokens=True,  # 添加特殊符号
        )
        input_ids.append(encoded_sent)
    return input_ids

input_ids = vectorize(train_data, tokenizer=bert_tokenizer)
input_ids[900]

[101,
 3172,
 7027,
 1065,
 1305,
 2593,
 3836,
 1912,
 769,
 6956,
 7270,
 1184,
 2518,
 2349,
 1825,
 3172,
 1788,
 102]

In [8]:
MAX_LEN = max([len(sen) for sen in input_ids])
print("Max sentence length: ", MAX_LEN)

Max sentence length:  35


In [9]:
# 填充成相同的长度
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("Padding token: {:}, ID: {:}".format(bert_tokenizer.pad_token,
                                           bert_tokenizer.pad_token_id))
input_ids = pad_sequences(input_ids,
                          maxlen=MAX_LEN,
                          dtype='long',
                          value=0,
                          truncating="post",
                          padding='post')
input_ids.shape,input_ids[900]

Padding token: [PAD], ID: 0


((180000, 35),
 array([ 101, 3172, 7027, 1065, 1305, 2593, 3836, 1912,  769, 6956, 7270,
        1184, 2518, 2349, 1825, 3172, 1788,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]))

In [10]:
# 对应的掩码
# 填充对应的mask

def create_mask(input_ids):
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)
    return attention_masks

mask = create_mask(input_ids)
print(mask[900])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
# 转化为 PyTorch 数据格式
import torch

train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(train_labels)
train_mask = torch.tensor(mask)
train_inputs.shape, train_labels.shape, train_mask.shape

(torch.Size([180000, 35]), torch.Size([180000]), torch.Size([180000, 35]))

In [13]:
# 创建数据管道

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
train_data = TensorDataset(train_inputs, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,
                              sampler=train_sampler,
                              batch_size=batch_size)
train_dataloader

In [14]:
len(train_dataloader)

5625

In [24]:
# 将整个过程整合起来


class DataGen:
    def __init__(self, path, tokenizer):
        self.tokenizer = tokenizer

        train_inputs, train_labels = self.load_corpus(path)
        self.train_labels = train_labels

        train_inputs = self.vectorize(train_inputs, self.tokenizer)
        MAX_LEN = max([len(seq) for seq in train_inputs])

        train_inputs = pad_sequences(
            train_inputs,
            maxlen=MAX_LEN,
            dtype='long',
            value=0,
            truncating="post",
            padding='post',
        )
        train_mask = self.create_mask(train_inputs)
        self.train_inputs = torch.tensor(train_inputs)
        self.train_labels = torch.tensor(train_labels)
        self.train_mask = torch.tensor(train_mask)

    def __call__(self, batch_size):
        train_data = TensorDataset(
            self.train_inputs,
            self.train_mask,
            self.train_labels,
        )
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(
            train_data,
            sampler=train_sampler,
            batch_size=batch_size,
        )
        return train_dataloader

    def load_corpus(self, path):
        sentences = []
        labels = []
        with open(path, 'r', encoding='UTF-8') as f:
            for line in tqdm(f):
                line = line.strip()
                if not line:
                    continue
                sent, label = line.split('\t')
                sentences.append(sent)
                labels.append(int(label))
        return sentences, labels

    def vectorize(self, sentences, tokenizer):
        input_ids = []

        for sent in sentences:
            encoded_sent = tokenizer.encode(
                sent,
                add_special_tokens=True,  # 添加特殊符号
            )
            input_ids.append(encoded_sent)
        return input_ids

    def create_mask(self, input_ids):
        attention_masks = []
        for sent in input_ids:
            att_mask = [int(token_id > 0) for token_id in sent]
            attention_masks.append(att_mask)
        return attention_masks


In [25]:
data_gen = DataGen(train_path, tokenizer=bert_tokenizer)
train_dataloader = data_gen(batch_size=32)

for data in train_dataloader:
    input_data, input_mask, input_labels = data
    print(input_data)
    print(input_mask)
    print(input_labels)
    break

180000it [00:00, 1415703.24it/s]


tensor([[ 101, 3119, 6397,  ...,    0,    0,    0],
        [ 101, 1649, 4448,  ...,    0,    0,    0],
        [ 101, 2797, 3952,  ...,    0,    0,    0],
        ...,
        [ 101,  683, 2157,  ...,    0,    0,    0],
        [ 101, 6205, 1298,  ...,    0,    0,    0],
        [ 101, 4263, 2894,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([2, 2, 8, 2, 9, 0, 2, 0, 0, 2, 1, 6, 6, 3, 8, 5, 6, 0, 8, 9, 1, 2, 7, 6,
        0, 4, 7, 8, 6, 5, 1, 8])


In [38]:
data_gen = DataGen(dev_path, tokenizer=bert_tokenizer)
validation_dataloader = data_gen(batch_size=32)

for data in validation_dataloader:
    input_data, input_mask, input_labels = data
    print(input_data)
    print(input_mask)
    print(input_labels)
    break

10000it [00:00, 1240441.25it/s]


tensor([[  101,  1093,   772,  1501,  4764,  3309,  7481,   707,  6444,  3146,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  4640,  7716,   977,  6756,  1767,   677,  4028,  2661,  7789,
           671,  2391,  5125,  4868,  4567,  2891,  2797,  6841,  3324,  4640,
          7716,  1199,  7339,  7270,   102,     0,     0,     0,     0,     0],
        [  101,  7674,  6963,  1278,  4906,  1920,  2110,  8166,  2399,  7770,
          5440,  2497,  1357,  5310,  3362,  3389,  6418,  5143,  5320,  2458,
          6858,   102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1921,  3823,  5428,  7032,  3959,  5401,  1863,  2270,  9111,
          2398,  6629,  5468,  2961,  1772,   817, 12075,  8129,   775,  8378,
          2835,   102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1849,  1164,  4408,  1762,  735

## 创建模型

In [35]:
model = BertForSequenceClassification.from_pretrained(
    '../../H/models/huggingface/bert-base-chinese',  # 本地文件载入
    num_labels=10,
    output_attentions=False,
    output_hidden_states=False,
)
for param in model.parameters():
    param.requires_grad = True
    
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [37]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# 优化器
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


# 学习率规划
from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [39]:

# 预测精度
import numpy as np


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 格式化时间显示
import time
import datetime


def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [47]:
import random


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed_val = 42
num_epochs = 2

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch in range(0, num_epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, num_epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 1000 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(
                step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)
        # 返回 (loss,logits,hidden_state,attentions) 元组

        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)

    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    for batch in validation_dataloader:

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():

            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy / nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch 1,000  of  5,625.    Elapsed: 0:01:28.
  Batch 2,000  of  5,625.    Elapsed: 0:02:57.
  Batch 3,000  of  5,625.    Elapsed: 0:04:26.
  Batch 4,000  of  5,625.    Elapsed: 0:05:55.
  Batch 5,000  of  5,625.    Elapsed: 0:07:24.

  Average training loss: 0.07
  Training epcoh took: 0:08:20

Running Validation...
(tensor([[-0.9750, -0.9032, -1.0428, -1.2282, -1.8091, -1.1258, -1.4433, 10.6276,
         -1.0413, -1.4416],
        [ 9.4536, -0.1385,  0.4910, -1.8662, -0.9697, -1.4404, -2.0414, -1.5558,
         -1.6219, -1.8161],
        [-1.5756, -0.9316, -1.4159, -0.6750, -1.1955, -1.2734, -2.1328, -0.1801,
         -1.4325,  9.7992],
        [ 8.9105, -1.1229,  0.2732, -1.3357,  0.3308, -1.4881, -1.9632, -1.4422,
         -2.0192, -1.9859],
        [ 9.1201, -1.1285,  1.0052, -1.8047,  0.0593, -1.5756, -2.3194, -1.4749,
         -1.7452, -1.9377],
        [ 1.8843,  8.9545, -1.2000, -0.8432, -2.4492, -1.0828, -0.0362, -1.2505,
         -

(tensor([[ 5.1261,  1.1557,  5.0805, -3.6128,  1.2693, -2.6066, -1.4499, -2.7093,
         -2.9051, -2.8216],
        [ 0.2735,  0.6618,  8.9383, -2.7442, -0.2859, -1.6692, -1.7893, -2.2619,
         -1.8428, -1.8879],
        [-1.0824, -1.3471, -0.2617, -1.1427, -1.6825, -1.1703, -0.8613, 10.2826,
         -1.3380, -1.4640],
        [-1.3557, -1.0721, -1.0243, -0.8253, -1.0093, -0.3858,  9.8834, -1.2650,
         -1.5949, -0.9276],
        [-0.9394, -0.7933, -1.9942, 10.0387, -1.2387, -0.4629, -0.8100, -1.3999,
         -1.2540, -0.3405],
        [-0.8523, -1.2019, -1.0564, 10.2829, -1.7956, -1.0182, -0.6048, -1.1769,
         -0.8209, -1.0694],
        [-0.5252, -0.4917, -1.9085, -0.1552,  0.3460, -1.1178,  8.6665, -1.9582,
         -1.4863, -1.1723],
        [-2.1939, -1.1817, -0.9548, -1.4594,  0.9647, -1.1487, -1.3566, -1.2636,
          9.3669,  0.0387],
        [-0.4728,  0.5653,  8.9648, -2.3398,  0.4034, -2.1872, -1.8386, -2.4762,
         -1.2856, -1.9750],
        [-1.0551, 

(tensor([[-1.9168e-01,  1.0189e+01, -1.3950e+00, -1.3402e+00, -1.4206e+00,
         -1.1188e+00, -1.0341e+00, -8.7780e-01, -1.5277e+00, -9.9845e-01],
        [-8.0169e-01, -9.3854e-01, -8.4589e-01, -1.1665e+00, -1.6795e+00,
         -1.3462e+00, -1.3975e+00,  1.0502e+01, -9.8319e-01, -1.7063e+00],
        [-1.2984e+00, -1.1451e+00, -1.3700e+00, -9.3563e-01, -1.3476e+00,
         -1.3126e+00, -2.1390e+00,  1.0267e+00, -1.9423e+00,  9.5415e+00],
        [-1.0723e+00, -7.7108e-01, -7.6298e-01, -1.0244e+00, -5.8344e-01,
         -1.3740e+00,  9.8208e+00, -1.0172e+00, -1.7242e+00, -1.1806e+00],
        [-1.5470e+00, -1.3516e+00, -2.1107e+00,  9.2334e-01, -7.1960e-01,
          9.6140e+00, -2.9882e-01, -1.3498e+00, -6.4332e-01, -8.9871e-01],
        [ 9.0381e+00, -8.0108e-01,  1.2352e+00, -1.5964e+00, -1.5095e-02,
         -1.8108e+00, -2.0408e+00, -1.5929e+00, -2.3533e+00, -2.2924e+00],
        [-2.2689e+00, -1.3138e+00, -3.3731e-01, -1.0686e+00,  1.0133e+00,
         -1.4149e+00, -1.2866e+

(tensor([[-0.9673, -2.2465, -0.9163, -1.8792,  8.8773, -1.5512, -0.5083, -1.6086,
          0.2796, -1.9372],
        [-1.0243, -1.0900, -1.2932,  9.9998, -1.8007, -0.7412, -0.9021, -1.0699,
         -1.0525, -0.5707],
        [-0.2641, -0.0948, -1.5465, -0.7122, -0.8684,  9.7163, -0.7829, -1.7554,
         -1.1295, -1.2779],
        [-1.4781, -1.2270, -1.3913, -1.1262, -1.0024, -1.1212, -2.2514,  1.1016,
         -1.8822,  9.5198],
        [-1.1318, -0.9206, -1.9938, -0.2816, -1.0653,  9.9188, -0.2238, -1.1908,
         -1.0924, -0.8405],
        [ 9.1062, -0.1825, -0.5828, -1.4293, -1.2187, -0.5652, -1.8446, -0.9332,
         -1.7542, -1.1324],
        [-0.7365, 10.1736, -1.1950, -1.5133, -1.2287, -1.3586, -1.0011, -0.7618,
         -1.1627, -0.9560],
        [-0.9174, -1.2371, -1.1300, -1.1814, -1.5694, -1.2857, -1.2208, 10.6161,
         -1.1482, -1.4607],
        [-1.3241, -2.1081, -0.2101, -1.8940,  9.6309, -1.6575, -1.0558, -1.6454,
         -0.3801, -2.3379],
        [-1.6927, 

(tensor([[ 9.4299e+00, -3.5895e-01, -2.9501e-01, -1.6656e+00, -1.2004e+00,
         -8.0532e-01, -1.6459e+00, -1.2724e+00, -1.7782e+00, -1.5043e+00],
        [-1.1412e+00, -8.5371e-01, -1.1635e+00,  1.0268e+01, -1.8656e+00,
         -1.2197e+00, -6.4969e-01, -1.1888e+00, -4.1875e-01, -9.7583e-01],
        [ 3.3789e-01, -2.1720e+00, -7.2531e-02, -1.1901e+00,  8.8905e+00,
         -1.2189e+00, -1.4948e+00, -2.0710e+00, -5.1981e-01, -2.9851e+00],
        [-9.4788e-01, -7.2717e-01, -8.7591e-01, -1.8563e+00, -1.2456e+00,
         -9.5717e-01, -1.7729e+00, -7.3006e-02, -2.0151e+00,  9.5902e+00],
        [-5.1144e-02, -1.0202e+00,  9.4619e+00, -1.7178e+00, -8.0601e-01,
         -1.7365e+00, -1.0249e+00, -2.3128e+00, -1.1715e+00, -1.5587e+00],
        [-1.4198e+00, -8.8238e-01, -1.2145e+00,  1.0293e+01, -1.6933e+00,
         -1.2610e+00, -3.7840e-01, -1.3399e+00, -5.9520e-01, -7.6823e-01],
        [-7.4854e-01, -8.3803e-01, -1.7494e+00, -3.9402e-01, -1.1085e+00,
          9.8125e+00, -1.3392e-

...

(tensor([[-6.4394e-01,  1.0228e+01, -1.1641e+00, -1.5704e+00, -1.2969e+00,
         -1.0354e+00, -7.6597e-01, -8.5753e-01, -1.3471e+00, -1.1336e+00],
        [-2.2265e+00, -1.4109e+00,  1.3042e+00, -1.6806e+00,  2.4601e+00,
         -9.8659e-01,  7.0647e+00, -2.5703e+00, -1.4654e+00, -1.1434e+00],
        [ 2.2004e+00,  6.4574e-01,  7.1853e+00, -3.1151e+00, -9.6071e-01,
         -2.7622e+00,  2.2650e+00, -2.9206e+00, -2.7262e+00, -2.5146e+00],
        [ 9.4000e+00, -4.2309e-01, -5.8716e-03, -1.7074e+00, -1.0898e+00,
         -1.1697e+00, -1.7842e+00, -1.2715e+00, -1.5834e+00, -1.4881e+00],
        [ 9.3612e+00, -6.4487e-01,  1.5047e+00, -1.4093e+00, -1.4285e+00,
         -1.7673e+00, -2.2806e+00, -1.7095e+00, -1.3718e+00, -1.8127e+00],
        [-2.1305e+00, -1.0308e+00, -2.6844e-01, -9.9360e-01, -2.5596e-02,
         -1.0423e+00, -1.1712e+00, -1.0227e+00,  1.0135e+01, -5.7768e-01],
        [ 9.1700e+00, -2.8977e-01,  1.0372e+00, -2.1686e+00, -2.0008e+00,
         -7.8297e-01, -2.0414e+

(tensor([[-2.1006, -1.5191, -0.8480, -1.0196,  1.6498, -0.9471, -1.4693, -1.6748,
          9.9790, -0.8437],
        [-1.1670, -1.4098, -1.6646, -0.9354, -1.3017, -1.0459, -1.4691, 10.6483,
         -1.2532, -0.8687],
        [-1.5628, -1.2405,  1.0840, -3.4690,  6.1824,  1.3719,  2.8659, -2.9820,
         -0.0662, -3.1030],
        [-1.8383, -1.3499,  0.5646, -1.5951,  1.1386, -1.4774, -1.6429, -1.7360,
          9.6843, -0.5860],
        [-0.9353, -1.0529, -0.8820, -1.2575, -1.5616, -1.3946, -1.4649, 10.6119,
         -0.9080, -1.4084],
        [ 0.3246, -1.6156, -0.4387, -2.0785, -1.0786, -0.7260, -2.7938, -0.2787,
         -1.7952,  8.6295],
        [ 1.7839,  9.3085, -0.1131, -2.4466, -0.8796, -1.6661, -1.8909, -1.6054,
         -2.1056, -1.1343],
        [ 8.8980, -0.7872,  3.0152, -2.0254, -0.8861, -2.3778, -2.3954, -2.0871,
         -1.8915, -2.3724],
        [ 0.4374, -0.3926,  9.5037, -2.1517, -1.1475, -1.8185, -1.2484, -2.5264,
         -1.5817, -1.2926],
        [ 9.0649, 

(tensor([[-0.4157, 10.1534, -1.4280, -1.5356, -1.0170, -1.3432, -1.2117, -0.7212,
         -1.2256, -1.0551],
        [-0.6349, -1.1226, -0.3058, -0.8886,  0.7320, -1.5920,  9.4559, -1.2003,
         -2.4225, -1.6671],
        [-1.2772, -0.9444, -1.6251, 10.0739, -1.9852, -0.4952, -0.5101, -1.3273,
         -0.3746, -0.9455],
        [-1.1531, -2.4726,  2.0029, -2.5502,  8.4913, -1.2340, -0.8857, -2.1733,
          0.8696, -3.3332],
        [ 0.3356,  4.7037, -1.1851,  3.2679, -1.4484,  2.0227, -1.4516, -2.5522,
         -1.3342, -2.1277],
        [ 0.0860,  0.2447,  8.8278, -1.8489,  0.5804, -2.0875, -1.6414, -2.6461,
         -1.7926, -2.4923],
        [-1.4285, -1.5537, -1.2724,  0.6990, -0.7276, -0.7285,  9.5539, -1.3384,
         -1.3530, -1.3389],
        [-1.9339, -1.7335, -1.5516, -0.9988, -1.3741,  0.7507, -1.8395, -1.1256,
         -1.1283,  9.5362],
        [-1.7853, -1.0291, -2.1249,  0.0166, -1.0161, -0.8645, -1.8536, -0.1939,
         -1.6272,  9.7279],
        [ 2.4884, 

(tensor([[-7.6086e-01, -2.1610e+00, -1.2738e+00, -1.3609e+00,  9.7115e+00,
         -1.2971e+00, -7.3047e-01, -1.4464e+00, -9.1357e-01, -2.3096e+00],
        [-1.9900e+00, -1.0250e+00, -3.9662e-01, -1.0524e+00,  2.2338e-01,
         -8.0412e-01, -1.1642e+00, -1.3434e+00,  1.0177e+01, -6.6301e-01],
        [-1.2330e+00, -1.2897e+00, -1.2102e+00, -1.0895e+00, -1.8134e+00,
         -1.1459e+00, -1.2454e+00,  1.0696e+01, -8.3820e-01, -1.2042e+00],
        [-1.3424e+00, -8.3535e-01, -1.1993e+00, -1.5981e+00, -1.4510e+00,
         -5.6324e-01, -1.8926e+00,  1.1223e-01, -1.7563e+00,  9.8125e+00],
        [-1.3998e+00, -9.0296e-01, -1.3105e+00,  1.0322e+01, -1.9875e+00,
         -8.0548e-01, -7.8721e-01, -1.1791e+00, -8.2732e-01, -4.4796e-01],
        [-1.7647e+00, -1.3917e+00, -2.1679e+00,  9.9022e+00, -1.1868e+00,
         -8.3065e-03, -4.0286e-01, -1.5630e+00, -7.9647e-01,  1.2228e-01],
        [-1.0635e+00, -9.4195e-01, -1.9419e+00, -3.0231e-01, -9.7842e-01,
          9.9163e+00,  1.6078e-

KeyboardInterrupt: 

## 测试模型

In [ ]:
test_gen = DataGen(test_path, tokenizer=bert_tokenizer)
test_dataloader = data_gen(batch_size=32)

for data in test_dataloader:
    input_data, input_mask, input_labels = data
    print(input_data)
    print(input_mask)
    print(input_labels)
    break

In [ ]:
for batch in test_dataloader:

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    print(outputs)
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy

    nb_eval_steps += 1

print("  Accuracy: {0:.2f}".format(eval_accuracy / nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

# ERINE模型分类

In [69]:
ernie_tokenizer = BertTokenizer.from_pretrained(
    '../../H/models/huggingface/ERNIE/')

# model = BertForSequenceClassification.from_pretrained(
#     '../../H/models/huggingface/ERNIE/',  # 本地文件载入
#     num_labels=10,
#     output_attentions=False,
#     output_hidden_states=False,
# )
# for param in model.parameters():
#     param.requires_grad = True
    
# model.cuda()

In [70]:
len(ernie_tokenizer)

17964

In [71]:
print(bert_tokenizer.tokenize(train_data[9999]))

['温', '家', '宝', '：', '去', '年', '进', '出', '口', '总', '额', '2', '.', '2', '万', '亿', '美', '元']


In [72]:
print(ernie_tokenizer.tokenize(train_data[9999]))

['温', '家', '宝', '：', '去', '年', '进', '出', '口', '总', '额', '2', '.', '2', '万', '亿', '美', '元']


In [57]:
len(ernie_tokenizer)

17964